#AUTOMAÇÃO PARA EXTRAÇÃO DE NOTÍCIAS E CRIAÇÃO DE DATASETS COM INFORMAÇÕES SOBRE AS PRINCIPAIS COMMODITIES DO BRASIL 

Afim de estudos, foi criado este notebook com o intuito de extrair dados e criar datasets contendo informações relevantes sobre algumas das principais commodities do Brasil. A partir disso, é possível criar alguns experimentos com algoritmos de Machine Learning. 

Os dados textuais foram extraídos do site *https://www.noticiasagricolas.com.br*. Entre eles estão *data da notícia, link, manchete, matéria*. Dados númericos (*preço do dia em real, preço do dia em dolar*) foram extraídos do site *https://www.cepea.esalq.usp.br/br* que fornecem dados sobre o agronegócio no Brasil. 

Com este notebook é possível extrair os dados de todas as categorias de notícias do site **Notícias Agrícolas**, porém para a criação dos datasets finais é preciso verificar se as séries de preços da categoria desejada está disponível no **Cepea USP** para download.  

Alguns dos datasets criados estão disponíveis no repositório *https://github.com/luixmartins/Web-Scraping-and-Data-Mining*. 



> Descrição das colunas do dataset 


*   **date:** data em que foi publicada a notícia 
*   **link:** link para a publicação 
*   **Headine:** Manchete da máteria 
*   **text:** texto publicado 
*   **cepea_real:** preço do dia em real fornecido pelo *Cepea*
*   **cepea_dollar:** preço do dia em dolar fornecido pelo *Cepea* 
*   **percentage_real:** cálculo representando o aumento/baixa diário em real  
*   **percentage_dollar:** cálculo representando o aumento/baixa diário em dolar
*   **trend:** a atribuição é feita a partir do valor da percentage_real, caso      acima de 0.5% é atribuída uma tendência de alta no preço para o próximo dia
*   **tarjet:** afins de uso em algoritmos de Machine Learning, segue o mesmo propósito do atributo trend 





In [ ]:
#instalação e importação dos modulos utilizados 
!pip install OleFileIO-PL -q

import re 
import urllib3 
import OleFileIO_PL 
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup 

     |████████████████████████████████| 122kB 23.4MB/s 
  ERROR: Failed building wheel for olefile
  ERROR: Failed cleaning build dir for olefile


In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

global http 
http = urllib3.PoolManager()

In [ ]:
#função para extrair links da página principal de notícias 
def get_links(url):
  try: 
    page = http.request('GET', url)
  except:
    print('Erro na pagina principal')+
    

  soup = BeautifulSoup(page.data, 'lxml')
  links = soup.find('div', {'class': 'lista-wrapper middle'}).find_all({'a'})

  return links

In [ ]:
#scraping dos dados 
def scraping_commoditie(df_data, links, last_date = None):
  for link in links:
    aux = str(link.get('href'))
    url = 'https://www.noticiasagricolas.com.br' + aux

    try:
      page = http.request('GET', url)
    except:
      print('Erro ao acessar a página', url)
    
    soup = BeautifulSoup(page.data, 'lxml')
    text_date = soup.find('div', {'class': 'datas'})

    if text_date:
      date = text_date.get_text(strip=True).split(' ')[2]
      transformed_date = pd.to_datetime(date, format='%d/%m/%Y') #controlar período da extração

      if transformed_date.year != 2021 and date != last_date:
        last_date = date

        headline = soup.find('h1', {'class': 'page-title'})
        if headline:
          headline = headline.get_text(strip=True)
        else:
          headline = None

        article = soup.find('div', {'class': 'materia'}) 
        if article:
          article = article.find_all({'p'})
          text = ''

          for txt in article:
            text += ' '.join(txt.find_all(text=True)).strip()

          text = ' '.join(text.split(' '))
        else:
          text = None 
        
        df_data = df_data.append({
            'date': date,
            'link': url,
            'headline': headline,
            'news': text
        }, ignore_index=True)

  df_data.dropna(inplace=True)

  return df_data

In [ ]:
#download da serie de preços do cepea 
def get_prices(url_prices):
  try:
    document = http.request('GET', url_prices)
  except:
    print('Erro ao realizar download do arquivo')
    document = None
  
  if document is not None:
    ole = OleFileIO_PL.OleFileIO(document.data)
    df_prices = pd.read_excel(ole.openstream('Workbook'), skiprows=3)

    return df_prices

#merge nos datasets dos dados e preços, criação de novas colunas 
def set_datasets(df_data, df_prices):
  df_prices.rename(columns={'Data': 'date', 'À vista R$': 'cepea_real', 
                            'À vista US$': 'cepea_dollar'}, inplace=True)
  df = pd.merge(df_data.reset_index(drop=True), df_prices, on='date', how='left')

  df.dropna(inplace=True)

  df = df.assign(percentage_real=np.nan, percentage_dollar=np.nan,
                trend=np.nan, tarjet=np.nan)
  
  return df

In [ ]:
#calculo para as colunas de porcentagem 
def calculate_percentage(x, y):
  if x > y:
    return ((x - y) * 100) / y 
  elif y > x:
    return (((y - x) * 100) / y) * (-1)
  else:
    return 0

#setando atributos tarjet e trend 
def set_tarjet(df):
  for index in range(1, len(df)):
    #CALCULO REAL 
    previous, next = df.iloc[index - 1, 4], df.iloc[index, 4]

    percentage = calculate_percentage(previous, next)
    df.iloc[index - 1, 6] = percentage

    if percentage > 0.5:
      df.iloc[index - 1, 8], df.iloc[index - 1, 9] = 'Up', 1
    else:
      df.iloc[index - 1, 8], df.iloc[index - 1, 9] = 'Down', 0

    #CALCULO DOLAR
    previous, next = df.iloc[index - 1, 5], df.iloc[index, 5]

    percentage = calculate_percentage(previous, next)
    df.iloc[index - 1, 7] = percentage

  df.dropna(inplace=True)
  df['tarjet'] = df['tarjet'].astype('int32')

  return df

In [ ]:
'''
LINKS BASE PARA PAGINA PRINCIPAL DAS NOTICIAS E DOWNLOAD DO CSV PARA PREÇOS.
AS SÉRIES DE PREÇOS ESTÃO DISPONIVEIS NO SITE CEPEA USP. 

PARA AS SÉRIES DE PREÇO É PRECISO INSPECIONAR O ELEMENTO (SÉRIE DE PREÇOS) 
NO SITE "https://www.cepea.esalq.usp.br/br/indicador/'COMMODITIE.ASPX'"
'''

url = 'https://www.noticiasagricolas.com.br/noticias/milho/?limit=100000' 
url_prices = "https://www.cepea.esalq.usp.br/br/indicador/series/milho.aspx?id=77"

links = get_links(url) #LINKS PARA NOTICIAS

df_data = pd.DataFrame(index=['date', 'link', 'headlines', 'news']) #DATAFRAME BASE NOTICIAS
df_data = scraping_commoditie(df_data, links) #SCRAPING 

df_prices = get_prices(url_prices) #DOWNLOAD XLSX PREÇOS 

df = set_datasets(df_data, df_prices) #MERGE E ATRIBUIÇÃO DE NOVAS COLUNAS
df = set_tarjet(df) #DEFINIÇÃO DE TARJET 

#TRATAMENTO DE INCONSISTÊNCIAS NOS TEXTOS 

In [ ]:
#Lista com algumas palavras removidas para o melhor uso em algoritmos de classificação
#A exclusão delas é opcional. 

treat_list = ['\xa0', 'Por Carla Mendes', 'MUMBAI (Reuters) - ', 'ISTAMBUL (Reuters) - ', 
  'SÃO PAULO (Reuters) - ', 'Por Roberto Samora', 'Por Ana Mano', 'PEQUIM (Reuters) - ',
  'Por Nayara Figueiredo', 'WASHINGTON (Reuters) - ', 'Algodão em NY-', 
  'Por Aluisio Alves e Roberto Samora', 'Por José Roberto Gomes', 
  'LONDRES (Reuters) - ', 'SÃO PAULO  - ', 'PARIS (Reuters) - ']

for index in range(len(df)):  
  text = df.iloc[index, 3]
  for txt in treat_list:
    text = text.replace(txt, '')
  
  text = text.strip()
  df.iloc[index, 3] = text


df['percentage_real'] = df['percentage_real'].astype('float')
df['percentage_dollar'] = df['percentage_dollar'].astype('float')

In [ ]:
df.to_excel('corn2015_2020.xlsx', index=False, encoding='utf-8-sig')

In [ ]:
df

,date,headline,link,news,cepea_real,cepea_dollar,percentage_real,percentage_dollar,trend,tarjet
0,30/12/2020,Preço do milho fecha 2020 em alta no Brasil; p...,https://www.noticiasagricolas.com.br/noticias/...,A quarta-feira (30) chega ao final com os preç...,78.65,15.17,0.254939,0.264375,Down,0
1,29/12/2020,Milho: demandas pontuais e volumes pequenos im...,https://www.noticiasagricolas.com.br/noticias/...,A terça-feira (29) chega ao final com os preço...,78.45,15.13,0.551141,1.611820,Up,1
2,28/12/2020,Milho se valoriza no Brasil e fecha a 2ªfeira ...,https://www.noticiasagricolas.com.br/noticias/...,A segunda-feira (28) chega ao final com os pre...,78.02,14.89,0.541237,-0.267917,Up,1
3,23/12/2020,Milho: quarta-feira (23) de ganhos em Chicago ...,https://www.noticiasagricolas.com.br/noticias/...,A quarta-feira (23) chega ao fim com os preços...,77.60,14.93,0.336178,-0.267201,Down,0
4,22/12/2020,Milho: ganhos em Chicago com demanda forte dão...,https://www.noticiasagricolas.com.br/noticias/...,A terça-feira (22) chega ao fim com os preços ...,77.34,14.97,2.982690,2.114598,Up,1
...,...,...,...,...,...,...,...,...,...,...
1530,09/01/2015,"Milho: À espera do USDA, mercado fecha sessão ...",https://www.noticiasagricolas.com.br/noticias/...,A primeira semana de negociação de 2015 foi ne...,27.95,10.59,-0.320970,0.761180,Down,0
1531,08/01/2015,"Milho: Com queda no dólar, preço recua 3,39% e...",https://www.noticiasagricolas.com.br/noticias/...,A queda no dólar pressionou o preço praticado ...,28.04,10.51,-0.213523,1.252408,Down,0
1532,07/01/2015,Milho: Queda na produção de etanol nos EUA pes...,https://www.noticiasagricolas.com.br/noticias/...,Os principais vencimentos do milho negociados ...,28.10,10.38,-0.354610,-0.669856,Down,0
1533,06/01/2015,Milho: Frente à expectativa de safra recorde n...,https://www.noticiasagricolas.com.br/noticias/...,"Em uma sessão de extrema volatilidade, os futu...",28.20,10.45,-1.225919,-0.570885,Down,0
